## **Environment Setup**
Run all when initiating session

In [ ]:
# retrieve config file, if using colab (nothing should happen if you use Windows)
!cp './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/config.py' '.'
!mkdir data
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/data/' '.'
!mkdir models
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/models/' '.'
!mkdir technical
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/technical/' '.'
!mkdir trainers
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/trainers/' '.'
!mkdir utils
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/utils/' '.'

In [ ]:
from config import *                            # config.py
from matplotlib import pyplot as plt 
import os
%matplotlib inline

## **GCS Integration**
Run all when initiating session

In [ ]:
if isWindows():
  import utils.gcs_windows as gcs 
elif isColab():
  import utils.gcs_colab as gcs
else:
  raise NotImplementedError('OS is not supported yet')

if isColab():
  gcs.init()
# no need of buckets for this project

## **Model Environment Setup**

In [ ]:
### tf-side debug ###
# more info: https://github.com/tensorflow/tensorflow/issues/29931
import tensorflow as tf
temp = tf.zeros([2, 16, 16, 3])  # Or tf.zeros
tf.keras.applications.vgg16.preprocess_input(temp)
print('')

In [ ]:
from technical.accelerators import strategy
from data.pipeline import *
from trainers.cnn import train, load_checkpoint
from models.encoder import Encoder
from utils.generator_loading_utils import load_generator_checkpoint

In [ ]:
encoder = Encoder(strategy)

In [ ]:
##### Load saved models here #####
RUN = False
# Warning, using this format on colab may cause the "'NoneType' object has no attribute 'get'" bug
if RUN:
  last_epoch = 0
  encoder.load(strategy=strategy, fname="encoder" + "-e_" + str(last_epoch))
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
##### Load saved models here (checkpoint) #####
RUN = False
if RUN:
  load_checkpoint(encoder, strategy)
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
##### Create and load saved generator here (checkpoint) #####
# please put generator's saved checkpoint in OUTPUT/generator_checkpoints (GCS for colab)
RUN = True
if RUN:
  generator = load_generator_checkpoint(strategy)
  print(generator)
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
LATENT_SIZE = Encoder.LATENT_SIZE 
IMAGE_SHAPE = Encoder.IMAGE_SHAPE
IMAGE_SIZE = [IMAGE_SHAPE[0], IMAGE_SHAPE[0]]
print("Latent size: " + str(LATENT_SIZE))
print("Image shape: " + str(IMAGE_SHAPE))

In [ ]:
##### Prepare dataset #####
training_dataset, validation_dataset = get_dataset(generator, strategy, batch=True)

## **Pre-Train Testing**

In [ ]:
noise = tf.random.normal([1,LATENT_SIZE])
print(noise)

In [ ]:
## generator output
image = generator.model(noise, training=False)[0][:,:,:3]
plt.imshow(image)

In [ ]:
## encoder output
import numpy as np
encoded = encoder.model(np.expand_dims(preprocess(image, None)[0], 0))
print(encoded)

In [ ]:
## regeneration
image = generator.model(encoded, training=False)[0]
plt.imshow(image)

## **Training**

In [ ]:
EPOCHS = 200

if isColab():                   # some colab's tpu side bug
  import numpy as np
  np.rank = np.ndim
train(encoder, training_dataset, validation_dataset, EPOCHS, strategy, restore_checkpoint=False)

## **Saving**

In [ ]:
encoder.save(0, strategy)

## **Testing**

In [ ]:
# use pre-train testing for now

## **Experiments**
Just a playground for trying out codes, nothing related at all, do not execute